In [160]:
import os
import time
import cv2 as cv
import statistics
import numpy as np
from ultralytics import YOLO
import cProfile
import voila
from IPython.display import display, HTML, clear_output
import ipywidgets as widgets
import tempfile
import base64

red = (255, 0, 0)
green = (0, 255, 0)

In [161]:
dist = 4.191
H = 3.048
a = -9.81

In [162]:
notebook_dir = os.getcwd()
project_root = os.path.dirname(notebook_dir)
pt_path = os.path.join(project_root, os.path.join('models', 'yolo11m.pt'))
data_path = os.path.join(project_root, os.path.join('data', 'data.txt'))
video_path = os.path.join(project_root, os.path.join('UPLOAD VIDEO HERE', 'video.mp4'))

In [163]:
def rad_to_deg(angle):
    return(angle * 180 / np.pi)

In [164]:
basketball_diameter = 0.24

ball_widths = [] #contains the ball widths in each frame, in order
ball_heights = [] #contains the ball heights in each frame, in order
person_heights = [] #contains the total vertical distance from top of body (usually head or hand) to the feet in each frame, in order
person_max_y = [] #contains the location of the top of the body (usually head or hand) in pixels in each frame, in order
person_max_y_valid_frames = [] #contains all person tops for only the frames the ball appears
frame_max_y_tuples = [] #lists the ordered pair (frame_num, person_max_y)
person_min_y = [] #contains the location of the feet in pixels in each frame, in order
ball_positions = [] #contains the location of center of ball in pixels in each frame, in order
ball_bottom = [] #stores only the bottom of the ball
valid_frames = [] #lists in order all of the frames in which the ball was detected

In [165]:
#set up the widgets
uploaded_video_content = None

upload_button = widgets.FileUpload(accept='.mp4', multiple=False, description='Upload Video')
analyze_button = widgets.Button(description='Analyze Shot',disabled=True,button_style='success')
status_label = widgets.Label(value="Please upload a .mp4 video file here.")
progress = widgets.FloatProgress(value=0.0, min=0.0, max=100.0, description='Progress:', bar_style='info', orientation='horizontal')
video_display = widgets.Image(value=b'',format='jpeg', width=1280, height=720, layout=widgets.Layout(border='2px solid #ddd', min_height='360px'))
results_display = widgets.HTML()

output_area = widgets.Output()

In [166]:
def on_upload_change(change):
    global uploaded_video_content
    with output_area:
        clear_output(wait=True) 
        if upload_button.value:
            file_data_tuple = upload_button.value 
            if file_data_tuple and isinstance(file_data_tuple, tuple) and len(file_data_tuple) > 0:
                file_info = file_data_tuple[0] 
                uploaded_video_content = file_info['content']
                status_label.value = f"Video '{file_info['name']}' uploaded. Click Analyze Shot to start."
                analyze_button.disabled = False
                results_display.value = "<div style='padding:10px;'></div>"
                video_display.value = b''
            else:
                status_label.value = "Error: No valid file info found in upload."
                analyze_button.disabled = True
                uploaded_video_content = None
        else:
            uploaded_video_content = None
            status_label.value = "No video uploaded."
            analyze_button.disabled = True


upload_button.observe(on_upload_change, names='value')


ui = widgets.VBox([
    status_label,
    upload_button,
    analyze_button,
    progress,
    video_display,
    results_display,
    output_area
])
display(ui)

In [167]:
def analyze_shot(change):
    global ball_widths, ball_heights, person_heights, person_max_y, person_max_y_valid_frames
    global frame_max_y_tuples, person_min_y, ball_positions, ball_bottom, valid_frames
    global uploaded_video_content

    #text displayed during processing
    with output_area:
        clear_output(wait=True)
        status_label.value = "Processing video..."
        progress.value = 0
        analyze_button.disabled = True

        if uploaded_video_content is None:
            status_label.value = "Please upload a .mp4 video."
            progress.value = 0
            analyze_button.disabled = False 
            return
        try:
            with tempfile.NamedTemporaryFile(suffix='.mp4', delete=False) as tmp_file:
                tmp_file.write(uploaded_video_content)
                video_path = tmp_file.name
        except Exception as e:
            status_label.value = f"Error saving temporary file: {e}"
            progress.value = 0
            analyze_button.disabled = False
            return

        try:
            ball = YOLO(pt_path)
        except Exception as e: #this line probably won't be used
            status_label.value = f""
            os.unlink(video_path)
            analyze_button.disabled = False
            return

        cap = cv.VideoCapture(video_path)
        if not cap.isOpened():
            status_label.value = "Error opening file. Please be sure your file is a .mp4 video."
            os.unlink(video_path)
            analyze_button.disabled = False
            return

        fps = float(cap.get(cv.CAP_PROP_FPS))
        t_per_frame = 1/fps
        total_frames = int(cap.get(cv.CAP_PROP_FRAME_COUNT))

        min_size = 10
        min_person_size = 10
        ball_color = green
        person_color = red

        frame_num = 0
        last_update_time = time.time()

        #this displays the frame-by-frame analysis done by YOLO
        while True:
            ret, frame = cap.read()
            if not ret:
                break

            frame_num += 1
            progress.value = (frame_num / total_frames) * 100

            current_time = time.time()
            if current_time - last_update_time > 0.1 or frame_num % 3 == 0:
                display_frame = frame.copy()
                display_frame = cv.resize(display_frame, (640, 360))

                display_results = ball(display_frame, verbose=False)

                for display_result in display_results:
                    for box in display_result.boxes:
                        cls_id = int(box.cls)
                        conf = box.conf.item()
                        x1, y1, x2, y2 = map(int, box.xyxy[0])


                        #labels
                        if cls_id == 32 and conf > 0.05:
                            cv.rectangle(display_frame, (x1, y1), (x2, y2), green, 1)
                            cv.putText(display_frame, f"Ball ({conf:.2f})", (x1, y1-10), cv.FONT_HERSHEY_SIMPLEX, 0.5, green, 2)
                        elif cls_id == 0 and conf > 0.5: 
                            cv.rectangle(display_frame, (x1, y1), (x2, y2), red, 2)
                            cv.putText(display_frame, f"Person ({conf:.2f})", (x1, y1-10), cv.FONT_HERSHEY_SIMPLEX, 0.5, red, 2)

                _, buffer = cv.imencode('.jpeg', display_frame)                
                video_display.value = buffer.tobytes()
                last_update_time = current_time

            #this is the math and physics analysis of the raw data that YOLO drew from the video
            #This section should be slightly more robust than my previous method, but it still has many limitations that I will work on in the future.
            basketballs = []
            persons = []

            results = ball(frame, verbose=False)

            for result in results:
                for box in result.boxes:
                    cls_id = int(box.cls)
                    conf = box.conf.item()
                    x1, y1, x2, y2 = map(int, box.xyxy[0])
                    w, h = x2 - x1, y2 - y1

                    if cls_id == 32 and conf > 0.05:
                        if w > min_size and h > min_size:
                            basketballs.append((x1, y1, x2, y2, conf))

                    elif cls_id == 0 and conf > 0.5:
                        if w > min_person_size and h > min_person_size:
                            persons.append((x1, y1, x2, y2, conf))


            frame_height = frame.shape[0]

            if persons:
                persons.sort(key=lambda p: p[4], reverse=True)
                best_person = persons[0]
                x1, y1, x2, y2, conf = best_person
                
                person_heights.append(y2-y1)
                person_max_y.append(frame_height-y1)
                person_min_y.append(frame_height-y2)
                frame_max_y_tuples.append((frame_num, frame_height-y1))

            best_ball = None
            if basketballs:
                basketballs.sort(key=lambda b: b[4], reverse=True)
                best_ball = basketballs[0]

            if best_ball:
                x1, y1, x2, y2, conf = best_ball
                current_ball_pos = ((x1 + x2) / 2, (frame_height - y2 + frame_height - y1) / 2)
                ball_bottom_y = (frame_height - y2)

                ball_widths.append(x2-x1)
                ball_heights.append((frame_height-y1)-(frame_height-y2))
                ball_positions.append(current_ball_pos)
                ball_bottom.append(ball_bottom_y)
                valid_frames.append(frame_num)
        


        cap.release()
        os.unlink(video_path)  

        #in case no ball is detected
        if not valid_frames:
            status_label.value = "No basketball detected."
            results_display.value = "" 
            progress.value = 100
            analyze_button.disabled = False
        
            return

        ball_width_px = statistics.median(ball_widths)
        ball_height_px = statistics.median(ball_heights)
        avg_dimension = ball_width_px
        cf = basketball_diameter / avg_dimension

        min_height = min(person_min_y) if person_min_y else 0

        person_max_y_valid_frames = []
        person_max_y_dict = {fnum: pmy for fnum, pmy in frame_max_y_tuples}
        
        last_known_person_max_y = 0
        if frame_max_y_tuples:
            frame_max_y_tuples.sort() 
            last_known_person_max_y = frame_max_y_tuples[0][1]

        for frame_id in valid_frames:
            if frame_id in person_max_y_dict:
                person_max_y_valid_frames.append(person_max_y_dict[frame_id])
                last_known_person_max_y = person_max_y_dict[frame_id]
            else:
                person_max_y_valid_frames.append(last_known_person_max_y)
        
        if not person_max_y_valid_frames:
            status_label.value = "Could not detect a person. Ensure a person is visible throughout the shot."
            results_display.value = ""
            progress.value = 100
            analyze_button.disabled = False

            return
    


        # Find frame of release
        frame_of_release = None
        ind = -1
        

        for i in range(1, len(valid_frames)):
            if i < len(ball_bottom) and i < len(person_max_y_valid_frames):
               if ball_bottom[i] > person_max_y_valid_frames[i] and \
                  ball_bottom[i-1] <= person_max_y_valid_frames[i-1]:
                   frame_of_release = valid_frames[i-1]
                   ind = i-1
                
        
        if frame_of_release is None:
            status_label.value = "Could not determine release point. Ensure the person and ball are clearly visible at release."
            results_display.value = ""
            progress.value = 100
            analyze_button.disabled = False 
            return
   
        launch_height_tuple = ball_positions[ind]
        launch_height = (launch_height_tuple[1] - min_height) * cf #to meter
        launch_x = (launch_height_tuple[0]) * cf
      
        least_valid_frame = None
        for frame_id in valid_frames:
            if frame_id > frame_of_release:
                least_valid_frame = frame_id
                break

        if least_valid_frame is None:
            status_label.value = "Error"
            results_display.value = ""
            progress.value = 100
            analyze_button.disabled = False
            return

        # Find the index of least_valid_frame in valid_frames
        try:
            lvf_index = valid_frames.index(least_valid_frame)
        except ValueError:
            status_label.value = "Error"
            results_display.value = ""
            progress.value = 100
            analyze_button.disabled = False
            return

        ball_height_lvf_tuple = ball_positions[lvf_index]
        ball_height_lvf = (ball_height_lvf_tuple[1] - min_height) * cf
        ball_x_lvf = (ball_height_lvf_tuple[0]) * cf 


        vertical_disp = ball_height_lvf - launch_height
        horizontal_disp = ball_x_lvf - launch_x
        elapsed_time = (least_valid_frame - frame_of_release) * t_per_frame

        if elapsed_time <= 0:
            status_label.value = "Error: Elapsed time for calculation is zero or negative. Cannot determine velocity."
            results_display.value = ""
            progress.value = 100
            analyze_button.disabled = False
            return


        initial_vel_up = (vertical_disp - 0.5 * (a) * (elapsed_time)**2) / elapsed_time
        vel_horizontal = horizontal_disp / elapsed_time

        v_0 = np.sqrt((initial_vel_up)**2 + (vel_horizontal)**2)
        
        detected_angle = np.arctan(initial_vel_up / vel_horizontal)
        diff_h = H - launch_height
        
        #this is the optimal angle method in the paper
        def new_optimal_angle_method(h):
            beta_rad = np.arctan(h/dist)
            beta = rad_to_deg(beta_rad)
            return 45 + beta/2
        theta_optimal_rad = new_optimal_angle_method(diff_h)*np.pi/180
        theta_optimal_deg = round(new_optimal_angle_method(diff_h), 2)
        detected_angle_deg = rad_to_deg(detected_angle)

        #next, based on the optimal angle found, find the speed that corresponds with it
        #this expression for the launch speed can be derived with 2D kinematics
        optimal_launch_speed = round(np.sqrt((a*dist**2)/(2*(np.cos(theta_optimal_rad)**2)*(diff_h-dist*(np.tan(theta_optimal_rad))))),2)
           
        #print the results
        result_text = f"""
        <h3>Shot Analysis Results</h3>
        <p><b>Your launch angle:</b> {detected_angle_deg:.2f}°</p>
        <p><b>Your shooting speed:</b> {v_0:.2f} m/s</p><br>
        <p style='color:green;'><b>Optimal shot:</b> {optimal_launch_speed} m/s at an angle of {theta_optimal_deg}°</p>
        """

        result_text += "<p>Compare your shot to the optimal shot above.</p>"

        
        results_display.value = result_text
        progress.value = 100
        analyze_button.disabled = False

In [168]:
analyze_button.on_click(analyze_shot)